In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def treasury_data_retrieval():
    df = pd.read_csv('../data/us_treasury_rates.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', ascending=True, inplace=True)
    df = df.reset_index(drop=True)

    return df

df = treasury_data_retrieval()

n = len(df)
h = 3

flatten = pd.DataFrame()
for id in range(1, n):
    row = df.iloc[id:(id+h), 1:].stack().reset_index(drop=True).to_frame().T
    flatten = pd.concat([flatten, row], ignore_index=True)

for id in range(1, 14):
    flatten[f'y_{id}'] = df.iloc[:(n-1), id]

flatten['Date'] = df['Date']
flatten.dropna(inplace=True)
flatten.columns = flatten.columns.astype(str)
flatten.set_index('Date', inplace=True)

flatten.tail()

,0,1,2,3,4,5,6,7,8,9,...,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12,y_13
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-26,4.76,4.70,4.60,4.54,4.43,4.34,4.19,4.17,4.11,4.17,...,4.52,4.45,4.37,4.21,4.21,4.17,4.24,4.30,4.56,4.48
2024-11-27,4.76,4.69,4.58,4.52,4.42,4.30,4.13,4.10,4.05,4.10,...,4.54,4.43,4.34,4.19,4.17,4.11,4.17,4.25,4.52,4.44
2024-11-29,4.75,4.63,4.51,4.51,4.43,4.30,4.17,4.11,4.08,4.13,...,4.52,4.42,4.30,4.13,4.10,4.05,4.10,4.18,4.45,4.36
2024-12-02,4.66,4.56,4.49,4.48,4.40,4.27,4.17,4.13,4.11,4.17,...,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,4.46,4.36
2024-12-03,4.65,4.54,4.47,4.46,4.38,4.23,4.13,4.09,4.07,4.13,...,4.48,4.40,4.27,4.17,4.13,4.11,4.17,4.23,4.50,4.40


In [3]:
def train_mse():
    predictions = model(dataset['train_input']).squeeze()  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']).squeeze()  # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse

In [4]:
import numpy as np

# Parameters for the sliding window
window_size = 380  # Number of samples in the training window
test_size = 30     # Number of samples in the test set for each iteration

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': []}

# Prepare data
X, y = flatten.iloc[:, :-13], flatten.iloc[:, -13:]
n_inputs = X.shape[1]
n_outputs = y.shape[1]

# Sliding window cross-validation
for start in range(0, len(X) + 1 - window_size - test_size, test_size):
    # Define the train and test indices
    train_idx = range(start, start + window_size)
    test_idx = range(start + window_size, start + window_size + test_size)
    
    print(f"Sliding Window Iteration: Train[{start}:{start+window_size}] Test[{start+window_size}:{start+window_size+test_size}]")
    
    # Split data into train and test for the current window
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Prepare dataset for the current window
    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(dtype).to(device)
    
    # Initialize the model
    model = KAN(width=[n_inputs, 20, n_outputs], grid=3, k=2, seed=42, device=device)
    
    # Train the model and compute metrics
    results = model.fit(dataset, opt="Adam", lamb=0.0, lr=0.001, steps=500, metrics=(train_mse, test_mse))

    # Store the metrics
    fold_results['train_mse'].append(results['train_mse'][-1])
    fold_results['test_mse'].append(results['test_mse'][-1])

# Calculate average metrics across all windows
avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])

print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {avg_test_mse}")

Sliding Window Iteration: Train[0:380] Test[380:410]
checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/500 [00:00<?, ?it/s]

| train_loss: 1.14e-01 | test_loss: 6.36e-02 | reg: 9.59e+01 | : 100%|█| 500/500 [00:12<00:00, 40.18


saving model version 0.1
Sliding Window Iteration: Train[30:410] Test[410:440]
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.12e-01 | test_loss: 6.56e-02 | reg: 9.25e+01 | : 100%|█| 500/500 [00:12<00:00, 41.07


...

saving model version 0.1
Sliding Window Iteration: Train[90:470] Test[470:500]
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.04e-01 | test_loss: 2.20e-01 | reg: 9.04e+01 | : 100%|█| 500/500 [00:11<00:00, 42.06


saving model version 0.1
Sliding Window Iteration: Train[120:500] Test[500:530]
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 8.69e-02 | test_loss: 2.05e-01 | reg: 9.03e+01 | : 100%|█| 500/500 [00:11<00:00, 42.32


saving model version 0.1
Sliding Window Cross-Validation Results
Average Train MSE: 0.011100981627021769
Average Test MSE: 0.026332114270081773


In [5]:
from sklearn.metrics import mean_squared_error 

y_pred = model(dataset['test_input']).cpu().detach().numpy().flatten()
y_test = dataset['test_label'].cpu().detach().numpy().flatten()

mean_squared_error(y_pred, y_test)

np.float64(0.04191239915109833)

In [8]:
pd.DataFrame({
    'pred': y_pred,
    'real': y_test}).head(5)

,pred,real
0,4.760041,4.92
1,4.947244,4.82
2,4.920877,4.73
3,4.871707,4.65
4,4.548059,4.45


In [9]:
help(model.fit)

Help on method fit in module kan.MultKAN:

fit(dataset, opt='LBFGS', steps=100, log=1, lamb=0.0, lamb_l1=1.0, lamb_entropy=2.0, lamb_coef=0.0, lamb_coefdiff=0.0, update_grid=True, grid_update_num=10, loss_fn=None, lr=1.0, start_grid_update_step=-1, stop_grid_update_step=50, batch=-1, metrics=None, save_fig=False, in_vars=None, out_vars=None, beta=3, save_fig_freq=1, img_folder='./video', singularity_avoiding=False, y_th=1000.0, reg_metric='edge_forward_spline_n', display_metrics=None) method of kan.MultKAN.MultKAN instance
    training

    Args:
    -----
        dataset : dic
            contains dataset['train_input'], dataset['train_label'], dataset['test_input'], dataset['test_label']
        opt : str
            "LBFGS" or "Adam"
        steps : int
            training steps
        log : int
            logging frequency
        lamb : float
            overall penalty strength
        lamb_l1 : float
            l1 penalty strength
        lamb_entropy : float
            ent